# 既存の計算結果を読み込んで実行する

以下の流れで、既存のaction ID(パラメータ)とその評価値を読み込み、PHYSBO を実行することができます。

1. 外部ファイルを読み込み、既存のaction ID(パラメータ)と評価値を読み込む。
2. action ID(パラメータ)と評価値をPHYSBOに登録する。
3. PHYSBO から次に実行するパラメータを得る。

時間制限の関係上、PHYSBO をずっと開いたままにできないため、インタラクティブに実行できないといった場合に、利用することができます。

## 探索候補データの準備

これまでのマニュアルでも利用していたs5-210.csv を用いてガウス過程を実行します。

In [9]:
import physbo

import numpy as np


def load_data():
    A =  np.asarray(np.loadtxt('data/s5-210.csv',skiprows=1, delimiter=',') )
    X = A[:,0:3]
    t  = -A[:,3]
    return X, t

X, t = load_data()
X = physbo.misc.centering(X)

## simulator の定義

In [10]:
class simulator:
    def __init__( self ):
        _, self.t = load_data()
    
    def __call__( self, action ):
        return self.t[action]

## 事前に計算したデータの用意

上述のload_data関数では全てのXとtが格納されています。ここでは事前に計算したとして、actoin IDのリストをランダムに20個取得し、その評価値を得ます。

In [11]:
import random
calculated_ids = random.sample(range(t.size), 20)
print(calculated_ids)
t_initial = t[calculated_ids]

[14609, 6267, 15683, 7557, 4815, 1793, 8699, 17939, 5189, 12394, 6651, 9602, 1047, 1898, 13619, 16445, 8030, 6007, 2155, 17548]


## action ID(パラメータ)と評価値をPHYSBOに登録

policyの初期変数initial_dataにcalculated_idsとt\[calculated_ids\]をリストとして登録します。

In [12]:
# policy のセット 
policy = physbo.search.discrete.policy(test_X=X, initial_data=[calculated_ids, t_initial])

# シード値のセット 
policy.set_seed( 0 )

## PHYSBO から次に実行するパラメータを取得

ベイズ最適化を行い、次の候補点を得ます。

In [8]:
actions = policy.bayes_search(max_num_probes=1, simulator=None, score="TS", interval=0,  num_rand_basis = 5000)
print(actions, X[actions])

Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0 -th epoch marginal likelihood 11.060833585964595
50 -th epoch marginal likelihood 5.561009734112506
100 -th epoch marginal likelihood 2.1480684432568324
150 -th epoch marginal likelihood -0.1527423111894528
200 -th epoch marginal likelihood -1.752632762706206
250 -th epoch marginal likelihood -2.8946877640854076
300 -th epoch marginal likelihood -3.728762097404015
350 -th epoch marginal likelihood -4.351070912776635
400 -th epoch marginal likelihood -4.825494303893425
450 -th epoch marginal likelihood -5.19560111865651
500 -th epoch marginal likelihood -5.491744638300084
Done

[8599] [[ 1.28309838 -0.29277002 -0.28097574]]


得られた候補点について外部で計算を行い、ファイルにactionsとそのスコアを登録する。再びファイルを読み込み、ベイズ最適化を実行し次の候補点を得るというプロセスを繰り返すことで、ベイズ最適化を進めることができます。